In [37]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [38]:
# Import database tables from the local 'sqlite_db_pythonsqlite.db' file
engine = create_engine('sqlite:////Users/mj/Desktop/Springboard Bootcamp/Springboard Projects_MJ/Unit 08_Country Club/sqlite_db_pythonsqlite.db')
club_facility = pd.read_sql_query('SELECT * FROM Facilities', engine)
club_booking = pd.read_sql_query('SELECT * FROM Bookings', engine)
club_member = pd.read_sql_query('SELECT * FROM Members', engine)


In [39]:
# A list of facilities with a total revenue less than 1000
merged_df = pd.merge(club_booking, club_facility, left_on='facid', right_on='facid', how='inner')
merged_df['total_revenue'] = merged_df.apply(lambda row: row['slots'] * row['guestcost'] if row['memid'] == 0 else row['slots'] * row['membercost'], axis=1)
revenue_df = merged_df.groupby('name')['total_revenue'].sum().reset_index()
revenue_df_new = revenue_df[revenue_df['total_revenue'] < 1000].copy()
revenue_df_new.sort_values(by='total_revenue', inplace=True)
revenue_df_new



,name,total_revenue
6,Table Tennis,180.0
4,Snooker Table,240.0
3,Pool Table,270.0


In [41]:
# Produce a report of members and who recommended them 
club_member['recommendedby'].replace('', np.nan, inplace=True) # Missing values treatment 
club_member['recommendedby'].fillna(-1, inplace=True)
club_member['recommendedby'] = club_member['recommendedby'].astype('int64')
report_member = pd.merge(club_member, club_member, left_on='recommendedby', right_on='memid', suffixes=('_member', '_recommender'))
report_member['MemberName'] = report_member['surname_member'] + ' ' + report_member['firstname_member']
report_member['Recommender'] = report_member['surname_recommender'] + ' ' + report_member['firstname_recommender']
report_df = report_member[['MemberName', 'Recommender']].sort_values('MemberName')
report_df 


,MemberName,Recommender
10,Bader Florence,Stibbons Ponder
9,Baker Anne,Stibbons Ponder
11,Baker Timothy,Farrell Jemima
7,Boothe Tim,Rownam Tim
1,Butters Gerald,Smith Darren
14,Coplin Joan,Baker Timothy
20,Crumpet Erica,Smith Tracy
5,Dare Nancy,Joplette Janice
13,Genting Matthew,Butters Gerald
21,Hunt John,Purview Millicent


In [48]:
# Find the facilities with their usage by member, but not guests 
usage_member = pd.merge(club_booking, club_facility, left_on='facid', right_on='facid')
usage_member = usage_member[usage_member['memid'] != 0]
grouped_usage_member = usage_member.groupby('name').size().reset_index(name='MemberUsage')
grouped_usage_member.rename(columns={'name': 'Facility'}, inplace=True)
grouped_usage_member


,Facility,MemberUsage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [47]:
# Find the facilities usage by month, but not guests
usage_month = pd.merge(club_booking, club_facility, left_on='facid', right_on='facid')
usage_month = usage_month[usage_month['memid'] != 0]
usage_month['Month'] = pd.to_datetime(usage_month['starttime']).dt.month
grouped_usage_month = usage_month.groupby(['name', 'Month']).size().reset_index(name='member_usage')
grouped_usage_month.rename(columns={'name': 'Facility'}, inplace=True)
grouped_usage_month.sort_values(by=['Facility', 'Month']).reset_index(drop=True, inplace=True)
grouped_usage_month


,Facility,Month,member_usage
0,Badminton Court,7,51
1,Badminton Court,8,132
2,Badminton Court,9,161
3,Massage Room 1,7,77
4,Massage Room 1,8,153
5,Massage Room 1,9,191
6,Massage Room 2,7,4
7,Massage Room 2,8,9
8,Massage Room 2,9,14
9,Pool Table,7,103
